# Extracting Data from separate cases to one json case file, while filtereing only relevent metadata

In [ ]:
import json
import os
from datetime import datetime

def extract_cases(source_file, target_file):
    """
    Extracts filtered case data from a JSON input file and saves to target file,
    including only the newest decision for each case. Appends or updates cases if target exists.
    """
    # --- Load source JSON ---
    with open(source_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # --- Load or initialize target JSON ---
    if os.path.exists(target_file):
        with open(target_file, "r", encoding="utf-8") as f:
            try:
                existing_cases = json.load(f)
            except json.JSONDecodeError:
                existing_cases = {}
    else:
        existing_cases = {}

    def first_or_empty(value):
        """Return first element if list, otherwise value or empty string."""
        if isinstance(value, list):
            return value[0] if value else ""
        return value or ""

    def parse_json_str(value):
        """Safely parse a JSON string like '{"code":"x","label":"y"}'."""
        if isinstance(value, str):
            try:
                return json.loads(value)
            except json.JSONDecodeError:
                return {}
        return value if isinstance(value, dict) else {}

    def parse_date(date_str):
        """Parse a date string to datetime for sorting (fallback to None)."""
        if not date_str:
            return None
        try:
            return datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        except Exception:
            return None

    # --- Process each case ---
    for case_id, case_data in data.items():
        metadata = case_data.get("metadata", {})
        decisions = case_data.get("decisions", [])

        # --- Find the newest decision ---
        newest_decision = None
        newest_date = None
        for decision in decisions:
            dec_meta = decision.get("metadata", {})
            adoption_dates = dec_meta.get("decisionAdoptionDate", [])
            adoption_date = first_or_empty(adoption_dates)
            parsed_date = parse_date(adoption_date)
            if parsed_date and (newest_date is None or parsed_date > newest_date):
                newest_date = parsed_date
                newest_decision = decision

        # --- Extract decision details ---
        decision_label = ""
        if newest_decision:
            dec_meta = newest_decision.get("metadata", {})
            decision_types = dec_meta.get("decisionTypes")
            if isinstance(decision_types, list) and decision_types:
                dec_item = decision_types[-1]
                dec_obj = parse_json_str(dec_item)
                decision_label = dec_obj.get("label", "")

        # --- Handle case sectors ---
        sector_code = sector_label = ""
        case_sectors = metadata.get("caseSectors", [])
        if isinstance(case_sectors, list) and case_sectors:
            sec_item = parse_json_str(case_sectors[0])
            sector_code = sec_item.get("code", "")
            sector_label = sec_item.get("label", "")

        # --- Handle caseLegalBasis (always list for labels) ---
        legal_basis_list = metadata.get("caseLegalBasis", [])
        case_legal_basis_code = ""
        case_legal_basis_label = []

        if isinstance(legal_basis_list, list) and legal_basis_list:
            for lb_entry in legal_basis_list:
                lb_obj = parse_json_str(lb_entry)
                if lb_obj:
                    case_legal_basis_code = lb_obj.get("code", case_legal_basis_code)
                    label = lb_obj.get("label", "")
                    if isinstance(label, str) and label:
                        parts = [p.strip() for p in label.split("+")]
                        case_legal_basis_label.extend(parts)

        # --- Handle caseCompanies (always list, split by "/") ---
        companies_raw = metadata.get("caseCompanies", [])
        companies = []

        if isinstance(companies_raw, list) and companies_raw:
            for c in companies_raw:
                if isinstance(c, str):
                    # Split by "/" if multiple companies listed together
                    parts = [p.strip() for p in c.split("/") if p.strip()]
                    companies.extend(parts)
        elif isinstance(companies_raw, str):
            companies = [p.strip() for p in companies_raw.split("/") if p.strip()]

        # --- Flatten and build output ---
        filtered_case = {
            "caseInstrument": first_or_empty(metadata.get("caseInstrument")),
            "caseNumber": first_or_empty(metadata.get("caseNumberPart")),
            "caseTitle": first_or_empty(metadata.get("caseTitle")),
            "caseSectorsCode": sector_code,
            "caseSectorLabel": sector_label,
            "caseCompanies": companies,  
            "caseLegalBasisCode": case_legal_basis_code,
            "caseLegalBasisLabel": case_legal_basis_label,  
            "caseLastDecisionDate": first_or_empty(metadata.get("caseLastDecisionDate")),
            "caseInitiationDate": first_or_empty(metadata.get("caseInitiationDate")),
            "decisionLabel": decision_label
        }

        existing_cases[case_id] = filtered_case

    # --- Save result ---
    with open(target_file, "w", encoding="utf-8") as f:
        json.dump(existing_cases, f, ensure_ascii=False, indent=2)

    print(f"✅ Extracted {len(existing_cases)} cases and saved to '{target_file}' (newest decisions only).")



# Fills all Merger cases with a legal basis = Art. 105

In [4]:
def fill_merger_legal_basis(json_file, fill_value="Art. 105"):
    """
    Update all cases with caseInstrument == 'Merger' and null/empty caseLegalBasisLabel
    to a specified fill_value.
    """
    import json
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    count = 0
    for case in data.values():
        instrument = case.get("caseInstrument", "")
        basis_labels = case.get("caseLegalBasisLabel")
        is_basis_null = (
            basis_labels is None
            or (isinstance(basis_labels, list) and not basis_labels)
            or (isinstance(basis_labels, str) and (basis_labels.strip() == "" or basis_labels.strip().lower() == "null"))
        )
        if instrument == "Merger" and is_basis_null:
            case["caseLegalBasisLabel"] = [fill_value]
            count += 1

    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Filled {count} 'Merger' cases with missing caseLegalBasisLabel!")



In [5]:
extract_cases("case-data-M.json", "cases.json")

fill_merger_legal_basis("cases.json")

extract_cases("case-data-AT.json", "cases.json")

✅ Extracted 9814 cases and saved to 'cases.json' (newest decisions only).
✅ Filled 9814 'Merger' cases with missing caseLegalBasisLabel!
✅ Extracted 10548 cases and saved to 'cases.json' (newest decisions only).


In [25]:
import json

def list_unique_legal_basis(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    legal_bases = set()
    for case in data.values():
        labels = case.get("caseLegalBasisLabel", [])
        if isinstance(labels, str):  # Handle string accidentally present
            labels = [labels]
        for label in labels:
            if isinstance(label, str) and label.strip():
                legal_bases.add(label.strip())

    print(f"Total unique legal basis: {len(legal_bases)}")
    for b in sorted(legal_bases):
        print("-", b)

# Usage example:
list_unique_legal_basis("cases.json")

Total unique legal basis: 12
- Art 23(1)e Regulation 2003/1
- Art 258 TFEU (Ex 226 EC)
- Art 65 ECSC Treaty
- Art. 101
- Art. 102
- Art. 105
- Art. 106
- Art. 37
- Art. 4 TFEU
- Art. 53
- Art. 54
- Art. 7 Reg.2003/1228


In [1]:
# to remove
# Art 23(1)e Regulation 2003/1
# - Art 258 TFEU (Ex 226 EC)
# - Art 65 ECSC Treaty
# - Art. 37
# - Art. 4 TFEU
# - Art. 53
# - Art. 54
# - Art. 37
# - Art. 7 Reg.2003/1228



# Normalizes legal basis variations

In [ ]:
import json

# Mapping of patterns to normalized values
mapping = {
    "Art. 101 TFEU": "Art. 101",
    "Art. 101": "Art. 101",
    "Art. 102 TFEU": "Art. 102",
    "Art. 102": "Art. 102",
    "Art. 105 TFEU": "Art. 105",
    "Art. 105 TFEU (Ex 85 EC)": "Art. 105",
    "Art 105 TFEU (Ex 85 EC)" : "Art. 105",
    "Art. 106": "Art. 106",
    "Art. 106 TFEU": "Art. 106",
    "Art. 106 TFEU (Ex 86 EC)": "Art. 106",
    "Art 106 TFEU (Ex 86 EC)" : "Art. 106",
    "Art. 53 EEA": "Art. 53",
    "Art. 53": "Art. 53",
    "Art. 54 EEA": "Art. 54",
    "Art. 54": "Art. 54"
}

def normalize_legal_basis(json_file, mapping, save_as=None):
    """
    For all cases in json_file, normalize `caseLegalBasisLabel` values according to mapping dict.
    Saves to the same file unless save_as is specified.
    """
    import json
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    for case_id, case_data in data.items():
        if "caseLegalBasisLabel" in case_data:
            labels = case_data["caseLegalBasisLabel"]
            # Defensive: sometimes might be string/null, not list
            if isinstance(labels, str):
                labels = [labels]
            case_data["caseLegalBasisLabel"] = [
                mapping.get(label.strip(), label.strip())
                for label in labels if isinstance(label, str) and label.strip()
            ]
    out_file = save_as or json_file
    with open(out_file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"✅ Legal basis normalization complete! Saved as {out_file}")





In [11]:
normalize_legal_basis("cases.json", mapping, save_as="cases.json")


✅ Legal basis normalization complete! Saved as cases.json


In [19]:
# list_unique_legal_basis("cases.json")

# Cleaning and parsing caseSectorLabel based on the NACE 

In [13]:
import json
import re

# NACE section names
NACE_SECTIONS = {
    "A": "AGRICULTURE, FORESTRY AND FISHING",
    "B": "MINING AND QUARRYING",
    "C": "MANUFACTURING",
    "D": "ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY",
    "E": "WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES",
    "F": "CONSTRUCTION",
    "G": "WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES",
    "H": "TRANSPORTATION AND STORAGE",
    "I": "ACCOMMODATION AND FOOD SERVICE ACTIVITIES",
    "J": "INFORMATION AND COMMUNICATION",
    "K": "FINANCIAL AND INSURANCE ACTIVITIES",
    "L": "REAL ESTATE ACTIVITIES",
    "M": "PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",
    "N": "ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES",
    "O": "PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY",
    "P": "EDUCATION",
    "Q": "HUMAN HEALTH AND SOCIAL WORK ACTIVITIES",
    "R": "ARTS, ENTERTAINMENT AND RECREATION",
    "S": "OTHER SERVICE ACTIVITIES",
    "T": "ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS- AND SERVICES-PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE",
    "U": "ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS AND BODIES"
}


def parse_sector_metadata(raw_sector: str):
    """Parse a raw sector label into structured sector metadata."""
    if not raw_sector or not isinstance(raw_sector, str):
        return None

    raw_sector = raw_sector.strip()

    # Split description
    parts = raw_sector.split(" - ")
    code_part = parts[0].strip()
    description = parts[1].strip() if len(parts) > 1 else None

    # Extract the section letter
    section_code = code_part[0] if code_part else None
    section_name = NACE_SECTIONS.get(section_code, None)

    # Extract numeric parts (e.g., 21, 20)
    numeric_parts = re.findall(r"\d+", code_part)
    division = numeric_parts[0] if len(numeric_parts) >= 1 else None
    group = numeric_parts[1][0] if len(numeric_parts) >= 2 else None
    class_code = numeric_parts[1][1:] if len(numeric_parts) >= 2 and len(numeric_parts[1]) > 1 else None

    return {
        "sectionCode": section_code,
        "sectionName": section_name,
        "division": division,
        "group": group,
        "classCode": class_code,
        "classDescription": description
    }


# === Load your JSON file ===
with open("cases.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# === Transform ===
for case_id, case_data in data.items():
    raw_sector = case_data.get("caseSectorLabel")
    sector_meta = parse_sector_metadata(raw_sector)
    case_data["sector_metadata"] = sector_meta

# === Save ===
with open("cases.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Done! Saved as cases.json")


✅ Done! Saved as cases.json


# Converting the Json to CSV

In [ ]:
import json
import pandas as pd
import numpy as np

# Load the JSON file
with open("cases.json", "r", encoding="utf-8") as f:
    data = json.load(f)

records = []
for case_id, case_data in data.items():
    # Get the nested metadata safely
    sector_meta = case_data.get("sector_metadata") or {}

    record = {
        "caseId": case_id,
        "caseInstrument": case_data.get("caseInstrument") or None,
        "caseNumber": case_data.get("caseNumber") or None,
        "caseTitle": case_data.get("caseTitle") or None,
        "caseSectorLabel": case_data.get("caseSectorLabel") or None,
        "caseLastDecisionDate": case_data.get("caseLastDecisionDate") or None,
        "caseInitiationDate": case_data.get("caseInitiationDate") or None,
        "decisionLabel": case_data.get("decisionLabel") or None,
        "sectionCode": sector_meta.get("sectionCode") or None,
        "sectionName": sector_meta.get("sectionName") or None,
        "division": sector_meta.get("division") or None,
        "group": sector_meta.get("group") or None,
        "classCode": sector_meta.get("classCode") or None,
        "classDescription": sector_meta.get("classDescription") or None,
        "caseCompanies": "; ".join(case_data.get("caseCompanies", [])) or None,
        "caseLegalBasisLabel": "; ".join(case_data.get("caseLegalBasisLabel", [])) or None,
    }

    records.append(record)

# Convert to DataFrame
df = pd.DataFrame(records)

# Replace empty strings with None
df.replace({"": np.nan}, inplace=True)

# Save to CSV
df.to_csv("cases.csv", index=False, encoding="utf-8")



✅ JSON successfully converted to cases.csv


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("cases.csv")

# Replace empty or invalid values with null
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.replace('NaN', np.nan)
df = df.replace('nan', np.nan)

# Save clean CSV
df.to_csv("cases.csv", index=False, na_rep="null")



✅ Clean CSV saved as cases_clean.csv with nulls for missing or blank values.


In [3]:
import pandas as pd

COMPANIES_TO_REMOVE = {"JV", "KKR", "CVC"}

def clean_companies(companies_str: str) -> str:
    if pd.isna(companies_str):
        return companies_str

    cleaned = []
    for comp in companies_str.split(";"):
        comp = comp.strip()
        if comp and comp.upper() not in COMPANIES_TO_REMOVE:
            cleaned.append(comp)

    return ";".join(cleaned)

# Load CSV
df = pd.read_csv("cases.csv")

# Clean company names
df["caseCompanies"] = df["caseCompanies"].apply(clean_companies)

df.to_csv("cases.csv", index=False)
